In [ ]:
from IPython.display import display
import pandas as pd

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [ ]:
report_dk = [{"value": 27, "product": "Subsistances", "destination": "fr"}, 
             {"value": 7, "product": "Subsistances", "destination": "colonies"}, 
             {"value": 37, "product": "Matières premières", "destination": "fr"}, 
             {"value": 70, "product": "Matières premières", "destination": "colonies"}, 
             {"value": 20, "product": "Drogues", "destination": "fr"}, 
             {"value": 29, "product": "Drogues", "destination": "colonies"}, 
            ]
my_other_source = [{"value": 27, "product": "Subsistances", "destination": "fr"}, 
             {"value": 7, "product": "Subsistances", "destination": "colonies"}, 
             {"value": 20, "product": "Matières premières", "destination": "fr"}, 
             {"value": 70, "product": "Matières premières", "destination": "other_field"}, 
             {"value": 40, "product": "Drogues", "destination": "fr"}, 
             {"value": 32, "product": "Drogues", "destination": "other_field"}, 
            ]

In [ ]:
for row in report_dk:
    row["group"] = "import"
for row in my_other_source:
    row["group"] = "export"
all_data = report_dk + my_other_source

In [ ]:
import json

with open("data/export_dunkerque_entreprôt.json", "r") as f:
    report_dk = json.load(f)

In [ ]:
all_data[:2]

In [ ]:
VegaLite({
  "title": "Répartition des exports depuis Dunkerque par destination",
  "data": {"values": all_data},
  "mark": {"type":"bar", "tooltip": {"content": "data"}},
  "encoding": {
    "x": {"field": "value", "type": "quantitative", "title": "Valeur (Lt)"},
    "column": {"field": "group"},
    "y": {"field": "product", "sort": "-x", "title": "Produits"},
    "color": {"field": "destination"}
  }
})

In [ ]:
VegaLite({
  "title": "Répartition des exports depuis Dunkerque par destination",
  "data": {"values": all_data},
  "spacing": 0,
  "hconcat": [{
    "transform": [{
      "filter": {"field": "group", "equal": "export"}
    }],
    "title": "export",
    "mark": "bar",
    "encoding": {
      "y": {
        "field": "product", "axis": None, "sort": "-x", "title": None
      },
      "x": {
        "field": "value",
        "type": "quantitative",
        "title": "Valeur (Lt)",
        "sort": "descending"
      },
      "color": {
        "field": "destination",
      }
    }
  }, {
    "transform": [{
      "filter": {"field": "group", "equal": "import"}
    }],
    "title": "import",
    "mark": "bar",
    "encoding": {
      "y": {
        "field": "product", "title": None,
        "axis": None, "sort": "-x"
      },
      "x": {
        "field": "value",
        "type": "quantitative",
        "title": "Valeur (Lt)"
      },
      "color": {
        "field": "destination",
      }
    }
  }]
}
)